In [ ]:
%load_ext bigquery_magics

# "Will it snow tomorrow?" - The time traveler asked
The following dataset contains climate information from over 9000 stations accross the world. The overall goal of these subtasks will be to predict whether it will snow tomorrow 20 years ago. So if today is 1 April 2025 then the weather we want to forecast is for the 2 April 2005. You are supposed to solve the tasks using Big Query, which can be used in the Jupyter Notebook like it is shown in the following cell. For further information and how to use BigQuery in Jupyter Notebook refer to the Google Docs. 

The goal of this test is to test your coding knowledge in Python, BigQuery and Pandas as well as your understanding of Data Science. If you get stuck in the first part, you can use the replacement data provided in the second part.

In [ ]:
%%bigquery
SELECT
*,
FROM `bigquery-public-data.samples.gsod`
LIMIT 20 


## Part 1

### 1. Task
Change the date format to 'YYYY-MM-DD' and select the data from 2000 till 2005 for station numbers including and between 725300 and 726300 , and save it as a pandas dataframe. Note the maximum year available is 2010. 

In [ ]:
%%bigquery df
SELECT *,
FORMAT_DATE('%Y-%m-%d', DATE(year, month, day)) AS date, #Add new column with new date format 
FROM `bigquery-public-data.samples.gsod`
WHERE year BETWEEN 2000 AND 2005 
    AND station_number BETWEEN 725300 AND 726300

In [ ]:
#Check the first rows of the created dataframe 
df.head()

### 2. Task 
From here you want to work with the data from all stations 725300 to 725330 that have information from 2000 till 2005. 

In [ ]:
%%bigquery df
SELECT *,
    FORMAT_DATE('%Y-%m-%d', DATE(year, month, day)) AS date,
FROM `bigquery-public-data.samples.gsod`
WHERE station_number BETWEEN 725300 AND 725330
    AND year BETWEEN 2000 AND 2005

In [ ]:
#Check general information of the new df
df.info()

Start by checking which year received the most snowfall in our data. 

In [ ]:
%%bigquery
SELECT year,
    COUNT(snow) AS nr_snowfall, #Count snowfall events
FROM `bigquery-public-data.samples.gsod`
WHERE station_number BETWEEN 725300 AND 725330
    AND year BETWEEN 2000 AND 2005
GROUP BY year 
ORDER BY nr_snowfall DESC #Order snowfall from largest to smallest
LIMIT 1 #Limit query result to the first row, which correspond to the highest number of snowfall events

Add an additional field that indicates the daily change in snow depth measured at every station. And identify the station and day for which the snow depth increased the most.  

In [ ]:
%%bigquery
WITH snow_depth_daily_change AS ( #Use temporary result query to simplify the process of next queries 
    SELECT station_number,
        FORMAT_DATE('%Y-%m-%d', DATE(year, month, day)) AS date,
        snow_depth,
        #Get the snow depth of previous day for the same station
        LAG(snow_depth, 1) OVER (PARTITION BY station_number ORDER BY year, month, day) AS prev_snow_depth,
        #Compute daily change in snow depth by subtracting the snow depth of the previous day from the current day at the same station
        snow_depth - LAG(snow_depth, 1) OVER (PARTITION BY station_number ORDER BY year, month, day) AS snow_depth_change
    FROM `bigquery-public-data.samples.gsod`
    WHERE station_number BETWEEN 725300 AND 725330
        AND year BETWEEN 2000 AND 2005
    )

#Get station and date with highest snow depth increase 
SELECT station_number,
    date,
    snow_depth_change
FROM snow_depth_daily_change #Take data from temporary query above 
WHERE snow_depth_change IS NOT NULL #Filter only not null snow depth changes
ORDER BY snow_depth_change DESC 
LIMIT 1

Do further checks on the remaining dataset, clean or drop data depending on how you see appropriate. 

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

pd.set_option("display.max.columns", None) #Display all columns of dataframes
pd.set_option("display.max.rows", None) #Display all rows of dataframes

#Check general info on the df and unique values for each column
df.info()
df.nunique()

#Check for duplicated rows and drop them
df.duplicated().sum()
df = df.drop_duplicates()

#Check for missing values in each columns
df.isnull().sum()

#Drop columns having missing values >50%, they can skew analysis and reduce accuracy of the ML model
#Inspect the potentially important columns which have a high number of missing values
df['snow_depth'].isna().groupby(df['station_number']).mean()
final_df = df.drop(['min_temperature','min_temperature_explicit', 'mean_station_pressure', 'num_mean_station_pressure_samples', 'snow_depth'], axis=1)
#Dropped snow_depth due to >90% missingness across all stations
#Handle missing values with imputation techniques 
#Impute numeric values for each station by using interpolation
final_df['max_gust_wind_speed'] = final_df.groupby('station_number')['max_gust_wind_speed'].transform(lambda x: x.interpolate(limit_direction='both'))
final_df['mean_sealevel_pressure'] = final_df.groupby('station_number')['mean_sealevel_pressure'].transform(lambda x: x.interpolate(limit_direction='both'))
final_df['num_mean_sealevel_pressure_samples'] = final_df.groupby('station_number')['num_mean_sealevel_pressure_samples'].transform(lambda x: x.interpolate(limit_direction='both'))

#Standardize the representation of missing values to avoid potential bugs 
final_df = final_df.mask(final_df.isna(), np.nan) #Replace all missing values with NaN from numpy

#Check stored data per station
#If necessary, drop stations with too little data
data_per_station = final_df['station_number'].value_counts()
#display(data_per_station)

#Sort dataframe based on station numbers and date for better clarity
df = df.sort_values(by=['station_number', 'date']).reset_index(drop=True)
#Convert column date from object type to datetime type
df['date'] = pd.to_datetime(df['date'])

#Check for gaps in consecutive dates, they can confuse the ML model
deltas = df['date'].diff()[1:] #Take the difference between consecutive dates
gaps = deltas[deltas > timedelta(days=1)] #Filter gaps greater than 1 day
#Print summary and details of each gap
print(f'{len(gaps)} gaps with average gap duration: {gaps.mean()}') 
for i, g in gaps.items(): 
    gap_start = df.loc[i - 1, 'date']
    print(f'Start: {gap_start.strftime("%Y-%m-%d")} | Duration: {g}')
#No relevant gaps are detected

#Save processed dataframe to csv in current folder
final_df.to_csv("final_df.csv", index=False)

### 3. Task
Now it is time to split the data, into a training, evaluation and test set. As a reminder, the date we are trying to predict snow fall for should constitute your test set.

In [ ]:
import datetime as dt

#The date we are trying to predict for
test_date = str(dt.datetime.today()- dt.timedelta(days=20*365)).split(' ')[0]
print(test_date)

In [ ]:
#Select all the dates before the test date to create training and evaluation set
train_eval_df = final_df[final_df['date'] < test_date]

#Select all the dates which correspond to the date we are trying to predict for
test_df = final_df[final_df['date'] == test_date]

#Split train/eval set to obtain one set for training and one set for validation
train_eval_df = train_eval_df.sort_values(by='date') #Sort set by date to maintain chronological order to avoid unrealistic forecasting
#Define an index for splitting the train/eval set (90% of data for training and 10% for validation)
split_idx = int(len(train_eval_df) * 0.9)
#Declare training set and evaluation set 
train_df = train_eval_df.iloc[:split_idx] #Takes the first rows till split index position 
eval_df = train_eval_df.iloc[split_idx+1:] #Takes the rows after split index position and one day before the test date

#Check the training and evaluation ranges to be sure they are correct and non-overlapping
print("Training range:", train_df['date'].min(), "to", train_df['date'].max())
print("Evaluation range:", eval_df['date'].min(), "to", eval_df['date'].max())

#Define final set for training, evaluating and testing the model
#Drop leakage-prone columns which indicate observed weather events, irrelevant columns (date) and target column (snow)
X_train = train_df.drop(columns=['fog', 'rain', 'snow', 'hail', 'thunder', 'tornado', 'date'])
X_eval = eval_df.drop(columns=['fog', 'rain', 'snow', 'hail', 'thunder', 'tornado', 'date'])
X_test = test_df.drop(columns=['fog', 'rain', 'snow', 'hail', 'thunder','tornado', 'date'])
y_train = train_df['snow']
y_eval = eval_df['snow']
y_test = test_df['snow']

## Part 2
If you made it up to here all by yourself, you can use your prepared dataset to train an algorithm of your choice to forecast whether it will snow on the following date for each station in this dataset:

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.utils import class_weight

#Compute class weights for unbalanced dataset
print(y_train.value_counts()) #Class of no snowfall has more samples than class snowfall
class_weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(enumerate(class_weights))

#Normalize the data to improve prediction
scaler = StandardScaler() #Create the scaler
X_train_scaled = scaler.fit_transform(X_train)  #Fit to training set and transform it
#Transform both evaluation and test set using same scale of training set
X_eval_scaled = scaler.transform(X_eval)      
X_test_scaled = scaler.transform(X_test)        

You are allowed to use any library you are comfortable with such as sklearn, tensorflow, keras etc. 
If you did not manage to finish part one feel free to use the data provided in 'coding_challenge.csv' Note that this data does not represent a solution to Part 1. 

In [ ]:
from sklearn.model_selection import GridSearchCV

#Test different parameters to feed the model for improving prediction
params = {'n_estimators': [100, 200], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [5, 10, None], 'criterion' :['gini', 'entropy']}
grid = GridSearchCV(RandomForestClassifier(random_state=42), params)
grid.fit(X_train_scaled, y_train)

#Print best parameters for the model based on training set
print("Best parameters:", grid.best_params_)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

#Train random forest model on training dataset to predict snowfall
RFC_model = RandomForestClassifier(random_state=42, 
                                   max_depth=5) #Tune the model based on GridSearchCV
RFC_model.fit(X_train_scaled, y_train)

In [ ]:
#Check model's performance by predicting on evaluation set
y_eval_pred = RFC_model.predict(X_eval_scaled)

#Generate report for the model
print("Evaluation Set Performance")
print(classification_report(y_eval, y_eval_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_eval, y_eval_pred))

In [ ]:
#Check model's perfomance on test set and generate report
y_test_pred = RFC_model.predict(X_test_scaled)

print("Final Test Set Performance")
print(classification_report(y_test, y_test_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_test_pred, labels=[0, 1]))

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix

#Train gradient boosting model to see if we can achieve better predictions 
XGBC_model = XGBClassifier(scale_pos_weight=class_weights[1]/class_weights[0], #Handle class imbalance
                           booster='gblinear', 
                           missing=np.nan) 
XGBC_model.fit(X_train_scaled, y_train)

In [ ]:
#Check model's performance by predicting on evaluation set
y_eval_pred = XGBC_model.predict(X_eval_scaled)

#Generate report for the model
print("Validation Set Performance")
print(classification_report(y_eval, y_eval_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_eval, y_eval_pred))

In [ ]:
#Check model's perfomance on test set and generate report
y_test_pred = XGBC_model.predict(X_test_scaled)

print("Final Test Set Performance")
print(classification_report(y_test, y_test_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_test_pred, labels=[0, 1]))